# ENGIE - GEMS 
## Evaluación de las habilidades de gestión de datos y digitalización. Conciliación datos

---
Prueba práctica del proceso de selección desarrollada por Alba Durán Merinero

## 1. Configuración de paths y librerías 

In [1]:
import funciones_caso_practico as f
import os

main_path = "datos/" 
facturas_path = main_path + "facturas_agiekey.xlsx"
mapping_path = main_path + "Mapping.xlsx"
deals_path = main_path + "Deals_Marzo_2024/"

## 2. Configuración de parámetros de la conciliación 

Es necesario establecer los criterios del análisis a llevar a cabo. Se necesita informar del Año, Mes, Origen y Servicio Facturado de las facturas a conciliar.
Toda la información debe aportarse en formato string (texto entrecomillado)

**IMPORTANTE:**
En este notebook se han configurado los parámetros para la reproducción de los resultados del apartado 2a.

In [2]:
#PARAMETROS PARA REPRODUCIR LOS RESULTADOS DEL APARTADO 2a
#Fecha
ano ='2024'
mes = '03'
#Origen y Servicio Facturado a conciliar, expresado igual que en el fichero Facturas_agiekey
origen = 'C.I. Almería'
servicio_facturado = 'Entrada PVB'

## 3. Fichero facturas 

In [3]:
#Carga del fichero de facturas
facturas = f.carga_xlsx(facturas_path)
print('Número total de facturas: ' + str(facturas.shape[0]))

Número total de facturas: 14


In [4]:
#Visualización df facturas inicial
facturas.head(2)

,NumeroFactura,Emisor,RazonSocialEmisor,DepEmisor,Receptor,RazonSocialReceptor,DepReceptor,FechaFactura,Importe,Moneda,Estado,FechaEstado,FechaRegistro,Destino,Contrato,Origen,ServicioFacturado,¿Verificada por BO?,Observaciones
0,2024264672,ESA86484292,"ENAGAS GTS, SAU",60,ESB82508441,ENGIE ESPAÑA SLU,NaN,4/3/24,123195.13,EUR,Recibida,4/3/24,4/3/24,eMail,344991,AVB,Servicio agregado AVB,NaN,NaN
1,2024264671,ESA86484292,"ENAGAS GTS, SAU",60,ESB82508441,ENGIE ESPAÑA SLU,NaN,4/3/24,111963.62,EUR,Recibida,4/3/24,4/3/24,eMail,344991,AVB,Servicio agregado AVB,NaN,NaN


### 3.1 Filtrado de facturas

In [5]:
#Filtro de Facturas según el Origen y Servicio Facturado a conciliar
facturas = facturas[(facturas['Origen'] == origen) & (facturas['ServicioFacturado'] == servicio_facturado)]
print('Número de facturas correspondientes al Origen y Servicio Facturado: ' + str(facturas.shape[0]))

Número de facturas correspondientes al Origen y Servicio Facturado: 1


In [6]:
#Visualización df facturas filtrando el concepto de Origen y Servicio Facturado correspondiente
facturas.head(2)

,NumeroFactura,Emisor,RazonSocialEmisor,DepEmisor,Receptor,RazonSocialReceptor,DepReceptor,FechaFactura,Importe,Moneda,Estado,FechaEstado,FechaRegistro,Destino,Contrato,Origen,ServicioFacturado,¿Verificada por BO?,Observaciones
2,2324004231,ESA86484334,"ENAGAS TRANSPORTE, SAU",NaN,ESB82508441,ENGIE ESPAÑA SLU,NaN,6/3/24,503763.47,EUR,Recibida,6/3/24,6/3/24,eMail,300289985,C.I. Almería,Entrada PVB,NaN,NaN


### 3.2 Resumen de facturas

In [7]:
#Resumen de facturas
resumen_fact = f.global_info(facturas)

#Prints de la información
print('Número de facturas correspondientes al Origen y Servicio Facturado: ' + str(resumen_fact['NumeroFacturasValidas'][0]))
print('Número de clientes: ' + str(resumen_fact['NumeroClientes'][0]))
print('Facturación global del concepto: ' + str(resumen_fact['FacturacionTotal'][0]))

Número de facturas correspondientes al Origen y Servicio Facturado: 1
Número de clientes: 1
Facturación global del concepto: 503763.47EUR


## 4. Carga del fichero de mapping 

In [8]:
#Carga fichero mapping
mapping = f.carga_xlsx(mapping_path)
#visualización df mapping inicial 
mapping.head(2)

,Origen,Servicio facturado,Portfolio,Commodity,DealType
0,C.I. Almería,Entrada PVB,MEDG2,IAC,GETRA
1,TVB,PLANTAS,JVLNG,ICC,GETRA


In [9]:
#Filtro de dataset Mapping según el Origen y Servicio Facturado a conciliar en Facturas
mapping = mapping[(mapping['Origen'] == origen) & (mapping['Servicio facturado'] == servicio_facturado)]
#Creación nueva columna
mapping = f.combcols(mapping, 'FileName', ['Portfolio','Commodity','DealType'])
#En caso de filas 100% duplicadas se mantiene solo la primera
mapping = mapping.drop_duplicates(keep='first')
#Visualización del df mapping filtrando el concepto de Origen y Servicio Facturado correspondiente
mapping.head(2)

,Origen,Servicio facturado,Portfolio,Commodity,DealType,FileName
0,C.I. Almería,Entrada PVB,MEDG2,IAC,GETRA,MEDG2_IAC_GETRA


### 4.1 Selección a través del mapping del archivo de Deals corresponiente para conciliar las facturas

In [10]:
#Lista de archivos que coinciden según el origen
file_names = list(mapping['FileName'])
#Obtencion de lista de archivos en el directorio de los deals
list_files = os.listdir(deals_path)
#Filtro de archivos que correspondientes
filtered_files = [file for file in list_files if (file.startswith(file_names[0])) & (file.endswith('.xlsx'))]
#Construcción del path
fil_file_path = deals_path + filtered_files[0]


## 5. Carga fichero de deals

In [11]:
deals = f.carga_xlsx(fil_file_path)
print('Numero total de deals: '+ str(deals.shape[0]))
#Visualización df deals inicial
deals.head(2)

Numero total de deals: 2


,Unnamed: 0,AggregatedKey,ApplicationPeriod,AuctionType,BookingSource,BOValidated,BOValidationDate,BOValidator,CancelsDealId,Capacity,...,TraderCode,TraderName,TradeType,TradingPlace,TradingSystem,Underlying,UnwindStatus,VAI,VersionCreationDate,VersionUpdateDate
0,0,"37591650,2024/05/17 09:15:39,GETRA",CAL24,NaN,MET,True,2024-03-11T13:26:05+01:00,GGH410,0,31500,...,D12\EF2015,AUBIN Baptiste,Market,MA,ME,IAC,Normal,0,2024-05-17T09:15:39+02:00,9999-01-01T00:00:00+01:00
1,1,"37056084,2024/02/09 14:35:58,GETRA",CAL23,NaN,MET,True,2024-02-09T14:35:58+01:00,GGH410,0,31500,...,D12\EF2015,AUBIN Baptiste,Market,MA,ME,IAC,Normal,0,2024-02-09T14:35:58+01:00,9999-01-01T00:00:00+01:00


### 5.1 Selección de Deals del año de estudio

In [12]:
#Filtramos los deals correspondientes al año actual
deals['StartDeliveryDate'] = deals['StartDeliveryDate'].astype(str)
deals['EndDeliveryDate'] = deals['EndDeliveryDate'].astype(str)
deals = deals[(deals['StartDeliveryDate'].apply(lambda x: x.startswith(ano)) & (deals['EndDeliveryDate'].apply(lambda x: x.startswith(ano))))]
print('Numero de Deals del año de estudio: ' + str(deals.shape[0]))
#Visualización de df deals filtrando los datos del año de estudio
deals.head(2)

Numero de Deals del año de estudio: 1


,Unnamed: 0,AggregatedKey,ApplicationPeriod,AuctionType,BookingSource,BOValidated,BOValidationDate,BOValidator,CancelsDealId,Capacity,...,TraderCode,TraderName,TradeType,TradingPlace,TradingSystem,Underlying,UnwindStatus,VAI,VersionCreationDate,VersionUpdateDate
0,0,"37591650,2024/05/17 09:15:39,GETRA",CAL24,NaN,MET,True,2024-03-11T13:26:05+01:00,GGH410,0,31500,...,D12\EF2015,AUBIN Baptiste,Market,MA,ME,IAC,Normal,0,2024-05-17T09:15:39+02:00,9999-01-01T00:00:00+01:00


### 5.2 Análisis de los sub-flows
La información de cada uno de los flows que se engloban en un Deal, se puede obtener del campo ['Flow'], el cual se encuentra en formato json

In [13]:
#Extracción de información de sub-flows de los registros correspondientes
flow_df = f.extract_df_from_json_src(df= deals, col= 'Flows',deal_col= 'Id')
print('Número total de sub-flows: ' + str(flow_df.shape[0]))
#Visualización df flows inicial
flow_df.head(2)

Generando dataframe para deal:  37591650
Concatenando dataframes generados...
Número total de sub-flows: 24


,Deal_Id,SubFlow_Id,Date,Subcantidad
0,37591650,93540225,2024-02-20T00:00:00+01:00,105279.586144
1,37591650,93540224,2024-02-20T00:00:00+01:00,409582.490000


In [14]:
#Selección de los flows del año y mes correspondientes a las facturas
flow_df = flow_df[flow_df['Date'].apply(lambda x: x.startswith(ano + '-' + mes))]
print('Número de sub-flows para análisis: ' + str(flow_df.shape[0]))
#Visualización df flows filtrando las fechas correspondientes al análisis
flow_df.head(2)

Número de sub-flows para análisis: 2


,Deal_Id,SubFlow_Id,Date,Subcantidad
2,37591650,93540227,2024-03-20T00:00:00+01:00,120318.957648
3,37591650,93540226,2024-03-20T00:00:00+01:00,383444.510000


## 6. Calculo de totales 
Calculos y operaciones necesarios para la obtención de información requerida por BackOffice

### 6.1 Facturas de AGIKEY

In [15]:
facturas[['NumeroFactura','Importe', 'FechaFactura']]

,NumeroFactura,Importe,FechaFactura
2,2324004231,503763.47,6/3/24


In [16]:
#Cantidades total de las facturas
TotalQuantityAGIKEY = facturas['Importe'].astype('float').sum()

### 6.2 Deals TRS

In [17]:
TotalQuantityTRS = flow_df['Subcantidad'].sum()

In [18]:
results_deals = deals[['Id', 'StartDeliveryDate', 'EndDeliveryDate']]
results_deals['Cantidad'] = TotalQuantityTRS
results_deals['Procedencia TRS'] = file_names[0]
results_deals = results_deals[['Id', 'Cantidad', 'StartDeliveryDate', 'EndDeliveryDate', 'Procedencia TRS']]
results_deals = results_deals.rename(columns={'Id':'Deal'})
results_deals

,Deal,Cantidad,StartDeliveryDate,EndDeliveryDate,Procedencia TRS
0,37591650,503763.467648,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,MEDG2_IAC_GETRA


In [25]:
flow_df

,Deal_Id,SubFlow_Id,Date,Subcantidad
2,37591650,93540227,2024-03-20T00:00:00+01:00,120318.957648
3,37591650,93540226,2024-03-20T00:00:00+01:00,383444.510000


### 6.2 Matcheo

In [20]:
Importe_conciliado = TotalQuantityTRS
Importe_no_conciliado = TotalQuantityAGIKEY - Importe_conciliado


## 7. Output final

In [21]:
print('('+ origen + ', ' + servicio_facturado +')')
print('---> FACTURAS DE AGIKEY <---')
facturas[['NumeroFactura','Importe', 'FechaFactura']].reset_index(drop=True)

(C.I. Almería, Entrada PVB)
---> FACTURAS DE AGIKEY <---


,NumeroFactura,Importe,FechaFactura
0,2324004231,503763.47,6/3/24


In [22]:
print('---> DEALS TRS <---')
results_deals

---> DEALS TRS <---


,Deal,Cantidad,StartDeliveryDate,EndDeliveryDate,Procedencia TRS
0,37591650,503763.467648,2024-01-01T00:00:00+01:00,2024-12-31T00:00:00+01:00,MEDG2_IAC_GETRA


In [33]:
deal_list = list(flow_df['Deal_Id'].unique())
for i in deal_list:
    aux_df = flow_df[flow_df['Deal_Id'] == i]
    aux_df = aux_df[['SubFlow_Id', 'Date', 'Subcantidad']].reset_index(drop=True)
    print(aux_df)

   SubFlow_Id                       Date    Subcantidad
0    93540227  2024-03-20T00:00:00+01:00  120318.957648
1    93540226  2024-03-20T00:00:00+01:00  383444.510000


In [24]:
print(' -----------MATCHEO-------------')
print('TotalQuantityAGIKEY: ' + str(TotalQuantityAGIKEY) + ' --> Facturación Total')
print('TotalQuantityTRS: '+ str(TotalQuantityTRS) + ' --> Importe Conciliado')
print('TotalQuantityAGIKEY - TotalQuantityTRS: ' + str(TotalQuantityAGIKEY-TotalQuantityTRS) + ' --> Importe No Conciliado')

 -----------MATCHEO-------------
TotalQuantityAGIKEY: 503763.47 --> Facturación Total
TotalQuantityTRS: 503763.467648 --> Importe Conciliado
TotalQuantityAGIKEY - TotalQuantityTRS: 0.002351999981328845 --> Importe No Conciliado
